In [100]:
import numpy as np

def gradient_descent_exact_line_search(A,b,xstart,tol=1e-5):
    print("Gradient descent")
    x = xstart.copy()
    while abs(A@x-b).max() > tol:
        g_c = A@x-b
        mu = np.dot(g_c,g_c)/np.dot(g_c,A@g_c)
        x = x - mu*(A@x-b) # por qué vale la pena no hacerlo por componentes? no hay inversassss!!!!!,1.1;!;!;!!:!:!::!
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

def Richardson(A,b,xstart,w,tol=1e-5):
    print("Richardson")
    x = xstart.copy()
    while abs(A@x-b).max() > tol:
        x = x + w*(b-A@x) 
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

def GaussSeidel_byrow(A,b,xstart,tol=1e-5):
    print("Gauss Seidel by row")
    x = xstart.copy()
    A_aux = A - np.eye(A.shape[0])*A 
    while abs(A@x-b).max() > tol:
        for i in range(A.shape[0]):
            x[i] = (1/A[i,i]) * (b[i] - np.dot(A_aux[i], x)) # np.sum([A_aux[i,j]*x[j] for j in range(A.shape[0])
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
def Jacobi_byrow(A,b,xstart,tol=1e-5): 
    print("Jacobi by row")
    x = xstart.copy()
    D = np.eye(A.shape[0])*A
    A_aux = A - D
    iteration_matrix = np.linalg.inv(D)@A_aux
    while abs(A@x-b).max() > tol:
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            x_new[i] = (1/A[i,i]) * (b[i] - np.dot((A_aux)[:,i],x)) 
        x = x_new.copy()
    print("Residual norm",np.linalg.norm(A@x-b))
    return x, iteration_matrix

def SOR_byrow(A,b,xstart,w,tol=1e-5):
    print("SOR by row")
    x = xstart.copy()
    A_aux = A - np.eye(A.shape[0])*A
    while abs(A@x-b).max() > tol:
        for i in range(A.shape[0]):
            x[i] = (1-w)*x[i] + w/A[i,i]*(b[i]- np.dot(A_aux[i], x)) #room for improvement
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

In [101]:
# test with 2D finite difference matrix
import numpy as np
n = 10
ufun2D = lambda x,y: np.sin(np.pi*x)*np.sin(np.pi*y)
f2D = lambda x,y: 8*np.pi**2*np.sin(2*np.pi*x)*np.sin(2*np.pi*y)
nx, ny, nz = n, n, 1
lx, ly = 1.0, 1.0 
dx, dy = lx/(nx), ly/(ny)
npoints = (nx ) * (ny ) * (nz ) 
x = np.linspace(0, lx , nx+1) 
y = np.linspace(0, ly , ny+1) 
z = [0]

Xplot = np.zeros((nx+1,ny+1,nz+1))
Yplot = np.zeros((nx+1,ny+1,nz+1))

Ix = np.eye(nx-1)
Iy = np.eye(ny-1)
A1Dx = np.diag(2.0*np.ones(nx-1)) + np.diag(-1.0*np.ones(nx-2),1)+ np.diag(-1.0*np.ones(nx-2),-1)
A1Dy = np.diag(2.0*np.ones(ny-1)) + np.diag(-1.0*np.ones(ny-2),1)+ np.diag(-1.0*np.ones(ny-2),-1)

A2D = np.kron(A1Dx, Iy) + np.kron(Ix,A1Dy)

Uhplot = np.zeros((nx+1, ny+1, 1 ))
USOL = np.zeros((nx+1, ny+1, 1 ))
b = np.zeros((nx+1,ny+1))

Dirichletnode =[]
k=0

for j in range(ny+1):
    for i in range(nx+1):
        Xplot[i,j,0] = x[i]
        Yplot[i,j,0] = y[j]
        USOL[i,j,0] = ufun2D(x[i],y[j])
        # Not on the boundary
        if i>0 and i <nx and j>0 and j<ny:
            b[i,j] = f2D(x[i],y[j])
        else:
            Dirichletnode.append(k)
        k+=1
b = b.reshape((nx+1)*(ny+1))
Intnodes= np.setdiff1d(range((nx+1)*(ny+1)), Dirichletnode)
uhsol1 = np.zeros((nx+1)*(ny+1))
uhsol2 = np.zeros((nx+1)*(ny+1))
uhsol3 = np.zeros((nx+1)*(ny+1))
uhsol4 = np.zeros((nx+1)*(ny+1))
uhsol5 = np.zeros((nx+1)*(ny+1))
uhsol6 = np.zeros((nx+1)*(ny+1))


## aquí queremos usar metodos iterativos :p 
uhsol1[Intnodes] = np.linalg.solve(A2D,dx**2*b[Intnodes])
uhsol2[Intnodes] = gradient_descent_exact_line_search(A2D,dx**2*b[Intnodes],np.ones_like(b[Intnodes]))
uhsol3[Intnodes] = Richardson(A2D,dx**2*b[Intnodes],np.ones_like(b[Intnodes]),0.01)
uhsol4[Intnodes] = GaussSeidel_byrow(A2D,dx**2*b[Intnodes],np.ones_like(b[Intnodes]))
uhsol5[Intnodes] = SOR_byrow(A2D,dx**2*b[Intnodes],np.ones_like(b[Intnodes]),1.5)
uhsol6[Intnodes], iteration_matrix = Jacobi_byrow(A2D,dx**2*b[Intnodes],np.ones_like(b[Intnodes]))

uhsol1 = uhsol1.reshape((nx+1,ny+1))
uhsol2 = uhsol2.reshape((nx+1,ny+1))
uhsol3 = uhsol3.reshape((nx+1,ny+1))
uhsol4 = uhsol4.reshape((nx+1,ny+1))
uhsol5 = uhsol5.reshape((nx+1,ny+1))
uhsol6 = uhsol6.reshape((nx+1,ny+1))




Gradient descent
Residual norm 3.6051555257934965e-05
Richardson
Residual norm 4.99879987183047e-05
Gauss Seidel by row
Residual norm 4.524744041100624e-05
SOR by row
Residual norm 2.878628039388593e-05
Jacobi by row
Residual norm 4.810672919512932e-05


In [102]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}],[{'is_3d': True},{'is_3d': True}],[{'is_3d': True},{'is_3d': True}]],
                    subplot_titles=['Numpy', 'Gradient Descent', 'Richardson', 'Gauss Seidel', 'SOR', 'Jacobi'])
                    
                
fig.add_trace(go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol1, colorscale='Viridis', ), 1, 1)
fig.add_trace(go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol2, colorscale='Viridis',), 1, 2)
fig.add_trace(go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol3, colorscale='Viridis', ), 2, 1)
fig.add_trace(go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol4, colorscale='Viridis', ), 2, 2)
fig.add_trace(go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol5, colorscale='Viridis', ), 3, 1)
fig.add_trace(go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol6, colorscale='Viridis', ), 3, 2)

fig.update_layout(title='Comparación de todos los métodos', autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90)
)


# Radio espectral de Jacobi, teórico vs aproximado

# Iteración alternativa para reducir el radio espectral

In [103]:

def Solve_alternative_iteration(n,b,xstart,tol=1e-5,its = 500):
    x = xstart.copy()
    En = np.diag(np.ones(n-1),1)+ np.diag(np.ones(n-1),-1)
    Mx = np.eye(n**2) * 4 - np.kron(np.eye(n), En)
    Nx = np.kron(En, np.eye(n))
    My = np.eye(n**2) * 4 - np.kron(En, np.eye(n))
    Ny = np.kron(np.eye(n),En)

    for i in range(its):
        x = np.linalg.solve(Mx, Nx@x + b)
        x = np.linalg.solve(My, Ny@x + b)
    
    return x, (np.linalg.inv(Mx)@Nx)@(np.linalg.inv(My)@Ny)
    
def spectral_radius(A):
    return np.max(np.abs(np.linalg.eigvals(A)))

uhsol7, A = Solve_alternative_iteration(n-1,dx**2*b[Intnodes].flatten(),np.ones_like(b[Intnodes].flatten()))

In [104]:
uhsol_test = np.zeros((nx+1)*(ny+1))
uhsol_test[Intnodes] = uhsol7
uhsol_test = uhsol_test.reshape((nx+1,ny+1))

fig = go.Figure(data=[go.Surface(x=Xplot[:,:,0], y=Yplot[:,:,0], z=uhsol_test, colorscale='Viridis')])
fig.update_layout(title='iteración alternativa', autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90)
)

In [105]:
jacobi_og_matrix = iteration_matrix
jacobi_alternative_matrix = A
jacobi_numerical_og_spectral_radius = spectral_radius(jacobi_og_matrix)
jacobi_numerical_alternative_spectral_radius = spectral_radius(jacobi_alternative_matrix)
jacobi_theorical_spectral_radius = lambda n1: (1/4)*(2*np.cos(np.pi/(n1+1))+2*np.cos(np.pi/(n1+1)))
jacobi_alternative_theorical_spectral_radius = lambda n1:(np.cos(np.pi/(2*(n1+1)))*np.cos(np.pi/(2*(n1+1))))/((2-np.cos(np.pi/(n1+1)))*(2-np.cos(np.pi/(n1+1))))

theorical_og_spectral_radius = jacobi_theorical_spectral_radius(n)
theorical_alternative_spectral_radius = jacobi_alternative_theorical_spectral_radius(n)
print("OG theorical spectral radius", theorical_og_spectral_radius)
print("OG numerical spectral radius", jacobi_numerical_og_spectral_radius)
print("Alternative theorical spectral radius", theorical_alternative_spectral_radius)
print("Alternative numerical spectral radius", jacobi_numerical_alternative_spectral_radius)


OG theorical spectral radius 0.9594929736144974
OG numerical spectral radius 0.9510565162951554
Alternative theorical spectral radius 0.9049481161394688
Alternative numerical spectral radius 0.8220693804387079


In [106]:
print("theorical alternative spectral radius", jacobi_alternative_theorical_spectral_radius(1000))
print("theorical og spectral radius", jacobi_theorical_spectral_radius(1000))

theorical alternative spectral radius 0.9999876877386734
theorical og spectral radius 0.9999950750566616
